# Computing covariate residuals
This workflow calculate the adjusted molecular phenotype data to adjust for additional covariates (i.e. kingship)

## Rational for covariate correction:

When we are doing xQtl analysis, typically, we performe variaous adjustment on both genotype matrix `X` as well as molecular phenotype matrix `Y`. This adjusting factors include sex, age, platform, etc. The overall goal of various adjustment is to rule out potential confounders in the analysis. 

However, when we are doing analysis, we may not always using the adjusted version of input data. Since `X` and `Y` are treated differerntly by differernt softwares, the corrected factors may not be identical. Specially, the correction in molecular phenotype matrix `Y` may be lesser that the genotype matrix `X`. This workflow implement additional covariate adjustment with regard to `Y`


## Method:

We regress out covariate `Z` out of `Y` by following methods: 

* Step 1: Regress `Y ~ Z`, get the estimated `βz`

* Step 2: Calculate `Y' = Y -  Z %*% βz` 

Since the output of step 2 is actually the residual of `Y` when regression `Z` out of `Y` , so our method is actually calculating the covariate residuals. 

Computationanlly, estimating `βz` needs `Z` to be full rank, result in `Z^T %*% Z` to be invertible. If `Z` is in low rank, we output an error.


## Input:

Required inputs are molecular phenotype data and covariate data. We use same input format as [PEER](https://cumc.github.io/xqtl-pipeline/pipeline/data_preprocessing/covariate/PEER_factor.html) and [APEX](https://cumc.github.io/xqtl-pipeline/pipeline/data_preprocessing/covariate/BiCV_factor.html) workflow. Please refer to their homepages for detail.

Optional input `--name` is the analysis name you want to include in the output file.

In this workflow, we do not need the samples in molecular phenotype data to be identical with covariate data. **Only same sample will be extracted and calculated**. This step using exact matching, so `Sample_1` and `sample_1` are **not treated as same**.

## Output

Output is a single file `{name}.mol_phe.resid.bed` with same format as the input molecular phenotype matrix. After that, the molecular phenotype data will be compressed by `bgzip` and a indexing file will be created. 

## Useage: 
```sos
    sos run remove_covariates.ipynb Residual_Y \
        --molecular_pheno_whole ... \
        --factor ... \
        ...
```

## Setup and global parameters

In [1]:
[global]
import os
# Work directory & output directory
parameter: wd = "."
# The filename name for output data
# parameter: container = 'gaow/twas'
# name for the analysis output
parameter: name = 'ROSMAP'
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
parameter: container = str

In [6]:
sos run remove_covariates.ipynb -h

usage: sos run remove_covariates.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  Residual_Y

Global Workflow Options:
  --wd '.'
                        Work directory & output directory
  --name ROSMAP
                        The filename name for output data parameter: container =
                        'gaow/twas' name for the analysis output
  --job-size 1 (as int)
                        For cluster jobs, number commands to run per job
  --walltime 5h
                        Wall clock time expected
  --mem 16G
                        Memory expected
  --numThreads 20 (as int)
                        Number of threads

Sections
  Residual_Y_1:         Get lm

In [7]:
# Get lm regression residual
[Residual_Y_1]
# Path to the input molecular phenotype data.
parameter: phenoFile = path
# Path to the factor file 
parameter: factor = path
input: phenoFile,factor
output: f'{wd}/{name}.mol_phe.resid.bed'
task: trunk_workers = 1, trunk_size = 1, walltime = '4h',  mem = '20G', tags = f'{step_name}_{_output[0]:bn}'
R: expand = "${ }", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout' , container = container

    library(dplyr)
    library(readr)

    pheno = read_delim(${_input[0]:r},delim = "\t")
    factor= read_delim(${_input[1]:r},delim = "\t")

    # Extract samples in both files
    extraction_sample_list <- intersect(colnames(pheno), colnames(factor)) 
    
    
    if(length(extraction_sample_list) == 0){
      stop("No samples are overlapped in two files!")
    }
    
    # Report identical samples:
    
    print("Listed samples are included in the analysis:")
    print(extraction_sample_list)
    
    # Subset the data:
    factor = factor[,extraction_sample_list]%>%as.matrix()%>%t()
    pheno_id = pheno%>%select(1:4)
    pheno = pheno%>%select(rownames(factor))%>%as.matrix()%>%t()
    
    # Get residual 
    pheno_resid = .lm.fit(x = factor, y = pheno)$residuals
    pheno_output = cbind(pheno_id, pheno_resid%>%t())
    pheno_output%>%write_delim(${_output[0]:r},delim = "\t")

# tabix via samtools
[Residual_Y_2]
output: f'{wd}/{name}.mol_phe.resid.bed.gz'
task: trunk_workers = 1, trunk_size = 1, walltime = '4h',  mem = '20G', tags = f'{step_name}_{_output[0]:bn}'
bash: expand = "${ }", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout', container = container
    bgzip -f ${_input}
    tabix -p bed ${_output}

## Minimum working example:

In [8]:
sos run remove_covariates.ipynb Residual_Y \
    --phenoFile PEER_example_data/Peer_example_data.bed \
    --factor PEER_example_data/Peer_example_cov.txt

INFO: Running Residual_Y_1: Get lm regression residual
INFO: Residual_Y_1 is completed.
INFO: Residual_Y_1 output:   ROSMAP.mol_phe.resid.bed
INFO: Running Residual_Y_2: tabix via samtools
INFO: Residual_Y_2 is completed.
INFO: Residual_Y_2 output:   ROSMAP.mol_phe.resid.bed.gz
INFO: Workflow Residual_Y (ID=w046e6e2411a33ea0) is executed successfully with 2 completed steps.
